<a href="https://colab.research.google.com/github/ChaoticMarauder/Deterministic_Model/blob/master/transport_matrix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()

Saving transport_data.xlsx to transport_data.xlsx


In [39]:
from google.colab import files
uploaded = files.upload()

Saving district_code.xlsx to district_code.xlsx


In [0]:
import pandas as pd
transport_df=pd.read_excel('transport_data.xlsx')

dataframe_collection = {} 


for i in range(11):
  dataframe_collection[i]=transport_df.iloc[:,2:5]
  transport_df.drop(transport_df.iloc[:,2:5], inplace = True, axis = 1) 

for i in range(11):
  dataframe_collection[i].columns=['District_code', 'District_name', 'trips']

temp_df=transport_df

final_df=transport_df
for i in range(10):
  final_df=pd.concat([final_df, temp_df], ignore_index=True) #merge vertically

trip_df=dataframe_collection[0]

for i in range(1,11):
  trip_df=pd.concat([trip_df, dataframe_collection[i]], ignore_index=True) #merge vertically

df = pd.concat([final_df, trip_df], axis=1) #merge horizontally

df.columns=['From_District_code', 'From_District_name','To_District_code', 'To_District_name', 'Trips']

df=df.dropna()
df['To_District_code']=df['To_District_code'].astype(int)

df['From_District_name']=df['From_District_name'].str.replace(" ","")
df['To_District_name']=df['To_District_name'].str.replace(" ","")
df

In [0]:
import pandas as pd
district_df=pd.read_excel('district_code.xlsx')
district_df.columns=['State_code', 'State_name', 'District_code', 'District_name']

district_df['District_name']=district_df['District_name'].str.replace(" ","")
district_df['State_name']=district_df['State_name'].str.replace(" ","")

state_array=district_df['State_name'].unique()
state_list=state_array.tolist()

district_array=district_df['District_name'].unique()
district_list=district_array.tolist()

state_group=district_df.groupby('State_name')

district_by_state_dict={}

for state in state_list:
  district_by_state_dict[state]=state_group.get_group(state)

maha=district_by_state_dict['Maharashtra']['District_code'].unique()

maha_list=maha.tolist()

maha_list

maha_district=district_by_state_dict['Maharashtra']['District_name'].unique()
maha_district_list=maha_district.tolist()



In [0]:
to_district_list=[]
from_district_list=[]
to_state_list=[]
from_state_list=[]

for index, row in df.iterrows():
  
  for index1,row1 in district_df.iterrows():
    
    if(row['To_District_code']==row1['District_code']):
      to_district_list.append(row1['District_name'])
      to_state_list.append(row1['State_name'])


    if(row['From_District_code']==row1['District_code']):
      from_district_list.append(row1['District_name'])
      from_state_list.append(row1['State_name'])

In [0]:
consolidated_df=pd.DataFrame(list(zip(from_state_list, to_state_list)), 
               columns =['From_State_name', 'To_State_name'])  

data_df = pd.concat([df, consolidated_df], axis=1, ignore_index=True)
data_df.drop(data_df.iloc[:, 0:2], inplace = True, axis = 1)
data_df.columns=['From_District_code', 'From_District_name','To_District_code', 'To_District_name', 'Trips','From_State_Name','To_State_Name']
data_df.to_csv('master_transport_file.csv', index=False)

In [0]:
from google.colab import files
files.download('master_transport_file.csv')

In [0]:
import numpy as np

group_1=data_df.groupby('From_State_Name')
maharashtra_df=group_1.get_group('Maharashtra')

group_2=maharashtra_df.groupby('To_State_Name')
maha_final_df=group_2.get_group('Maharashtra')


maha_tran_array = np.zeros(shape=(36,36))

for i in maha_list:
  for j in maha_list:
    for index, row in maha_final_df.iterrows():
      if(row['From_District_code']==i and row['To_District_code']==j):
        maha_tran_array[i-356][j-356]=row['Trips']


transport_maha_df = pd.DataFrame(maha_tran_array, index=maha_district_list, columns=maha_district_list)

In [135]:
maha_tran_array


array([[   0.,    0.,    0., ...,    0.,    0.,    0.],
       [   0.,    0., 7971., ...,    0., 7823.,    0.],
       [   0., 3603.,    0., ..., 2209., 2491., 2198.],
       ...,
       [   0.,    0., 3746., ...,    0.,    0., 3243.],
       [   0., 2360., 1663., ...,    0.,    0., 1440.],
       [   0.,    0., 2835., ..., 2467., 2782.,    0.]])

In [0]:
transport_maha_df.to_csv('maha_transport_matrix.csv', index=True)

In [0]:
from google.colab import files
files.download('maha_transport_matrix.csv')

In [141]:
from google.colab import files
uploaded = files.upload()

Saving master_transport_file.csv to master_transport_file (1).csv


In [0]:
import pandas as pd

master_df=pd.read_csv('master_transport_file (1).csv')
district_df=pd.read_excel('district_code.xlsx')
district_df.columns=['State_code', 'State_name', 'District_code', 'District_name']

district_df['District_name']=district_df['District_name'].str.replace(" ","")
district_df['State_name']=district_df['State_name'].str.replace(" ","")

state_array=district_df['State_name'].unique()
state_list=state_array.tolist()

state_code_array=district_df['State_code'].unique()
state_code_list=state_code_array.tolist()

feed_df=pd.DataFrame(list(zip(state_list, state_code_list)), 
               columns =['State_name', 'State_code'])  


In [0]:
to_state_code_list=[]
from_state_code_list=[]

for index, row in master_df.iterrows():
  
  for index1,row1 in feed_df.iterrows():
    
    if(row['To_State_Name']==row1['State_name']):
      to_state_code_list.append(row1['State_code'])


    if(row['From_State_Name']==row1['State_name']):
      from_state_code_list.append(row1['State_code'])

In [0]:
code_df=pd.DataFrame(list(zip(from_state_code_list, to_state_code_list)), 
               columns =['From_State_code', 'To_State_code'])  

finalised_df=pd.concat([master_df, code_df], axis=1)

finalised_df.to_csv('final_transport_file.csv', index=False)

In [0]:
from google.colab import files
files.download('final_transport_file.csv')

In [0]:

for index, row in finalised_df.iterrows():
  if(row['From_State_code']==row['To_State_code']):
    finalised_df=finalised_df.drop(index)

In [0]:
national_tran_array = np.zeros(shape=(37,37))

for i in state_code_list:
  for j in state_code_list:
    for index, row in finalised_df.iterrows():
      if(row['From_State_code']==i and row['To_State_code']==j):
        national_tran_array[i-1][j-1]+=row['Trips']

In [178]:
transport_national_df = pd.DataFrame(national_tran_array, index=state_list, columns=state_list)
transport_national_df.to_csv('India_Transportation_matrix.csv', index=True)

,AndamanandNicobar,AndhraPradesh,ArunachalPradesh,Assam,Bihar,Chandigarh,Chhattisgarh,DadraandNagarHaveli,DamanandDiu,Goa,Gujarat,Haryana,HimachalPradesh,JammuandKashmir,Jharkhand,Karnataka,Kerala,Ladak,Lakshadweep,MadhyaPradesh,Maharashtra,Manipur,Meghalaya,Mizoram,NCTofDelhi,Nagaland,Odisha,Puducherry,Punjab,Rajasthan,Sikkim,TamilNadu,Telangana,Tripura,UttarPradesh,Uttarakhand,WestBengal
AndamanandNicobar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AndhraPradesh,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,42109.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,56159.0,30375.0,0.0,0.0,0.0,52333.0,102531.0,0.0,0.0,0.0,0.0
ArunachalPradesh,0.0,0.0,0.0,19033.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,159.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Assam,0.0,0.0,10406.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16775.0,16483.0,5275.0,0.0,25275.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7435.0,0.0,0.0,16432.0
Bihar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,37807.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,50925.0,0.0,16584.0
Chandigarh,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,44125.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64983.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Chhattisgarh,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6179.0,0.0,0.0,0.0,0.0,16854.0,5480.0,0.0,0.0,0.0,0.0,0.0,14519.0,0.0,0.0,0.0,0.0,0.0,7400.0,0.0,1665.0,0.0,0.0
DadraandNagarHaveli,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11645.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11228.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
DamanandDiu,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33336.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Goa,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,28483.0,0.0,0.0,0.0,0.0,7145.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
from google.colab import files
files.download('India_Transportation_matrix.csv')